In [34]:
###Notes - Do 5 random summary schedules check every month and check totals
###Run 2X 
#1. archivedROs manually add comp,account,control batch from mcadcsv once.
#2. Of the same comp,account,control tach - oldest item >300 days and while newest item <210 days will stay on agedROs
#3. archivedROs can be removed and still good.
#4. in exel format cells, custom, add yyyy-mm-dd format style

In [35]:
#runSchedules

import os,datetime,pprint,logging,copy
import pandas as pd
import numpy as np
logging.basicConfig(level=logging.DEBUG, format=' %(asctime)s - %(levelname)s - %(message)s')


In [36]:
os.chdir(r'C:\Users\lxie\Desktop\Python Warranty') #work
from storeSchedulesDict import allStores
schedules = allStores
schedules

{'16': ['26300', '26410', '26310', '26330', '26320'],
 '17': ['26300', '26410', '26320'],
 '18': ['26300', '26410', '26420', '26320'],
 '19': ['26410', '26420'],
 '20': ['26300', '26410', '26320'],
 '23': ['26300', '26410'],
 '24': ['26300', '26410'],
 '25': ['26300', '26410', '26420'],
 '26': ['26300', '26410', '26320'],
 '29': ['26300', '26410', '26420', '26320'],
 '33': ['26300', '26410', '26320'],
 '34': ['26300', '26410', '26420', '26320'],
 '35': ['26300', '26410', '26420', '26320'],
 '38': ['26300', '26410', '26420', '26320'],
 '39': ['26300', '26410', '26310'],
 '40': ['26300', '26410', '26420'],
 '41': ['26300', '26410', '26320'],
 '42': ['26300', '26410', '26420'],
 '43': ['22700', '21200', '22800'],
 '44': ['26300', '26410', '26310', '26320'],
 '45': ['26300', '26410'],
 '47': ['26300', '26410', '26320'],
 '48': ['26300', '26410', '26320'],
 '49': ['26300', '26410'],
 '50': ['26300', '26410'],
 '60': ['22700', '21200', '22800'],
 '61': ['26300'],
 '62': ['26300'],
 '63': ['2

In [37]:
#Format Columns
thecolumns = ['Comp','Account','Jrnl','Reference','Date','Amount','Control1','Post-Date','Post-Time','User']
strcolumns = ['Comp','Account','Jrnl','Reference','Control1','User']
summary_columns = ['Comp','Account','Control1','Days','Amount','Current','31-60 days','61-90 days','91+']
detail_columns = ['Comp','Account','Control1','Jrnl','Days','Reference','Date','Amount','Current','31-60 days','61-90 days','91+']
Comp_Acct_Ctrl = ['Comp','Account','Control1']
total_columns = ['Reference','Amount','Current','31-60 days','61-90 days','91+']

In [38]:
#agedROs = pd.DataFrame(columns=thecolumns)
#agedROs.to_csv(r'C:\Users\lxie\Desktop\Python Warranty\agedROs.csv',index=False) #work

In [39]:
#UPLOAD DATA

mcadcsv = pd.read_csv(r'C:\ADP\SCHMCAD.csv',usecols=thecolumns,dtype=str)###work
combinedROs = pd.DataFrame(columns=thecolumns)
agedROs = pd.read_csv(r'C:\Users\lxie\Desktop\Python Warranty\agedROs.csv',usecols=thecolumns,dtype=str)###work
archiveROs = pd.read_csv(r'C:\Users\lxie\Desktop\Python Warranty\archivedROs.csv',usecols=Comp_Acct_Ctrl,dtype=str)###work
to_agedROs = pd.DataFrame(columns=thecolumns)
sch_details = pd.DataFrame(columns=thecolumns)
mcad_prep = pd.DataFrame(columns=thecolumns)

In [40]:
#Format Data

#Date to datetime
combinedROs['Date'] = pd.to_datetime(combinedROs['Date'])
mcadcsv['Date'] = pd.to_datetime(mcadcsv['Date'])
mcad_prep['Date'] = pd.to_datetime(mcad_prep['Date'])
agedROs['Date'] = pd.to_datetime(agedROs['Date'])
to_agedROs['Date'] = pd.to_datetime(to_agedROs['Date'])

#Post Date to datetime
combinedROs['Post-Date'] = pd.to_datetime(combinedROs['Post-Date'])
mcadcsv['Post-Date'] = pd.to_datetime(mcadcsv['Post-Date'])
mcad_prep['Post-Date'] = pd.to_datetime(mcad_prep['Post-Date'])
agedROs['Post-Date'] = pd.to_datetime(agedROs['Post-Date'])
to_agedROs['Post-Date'] = pd.to_datetime(to_agedROs['Post-Date'])

#Amount to 'non-null float64'
combinedROs['Amount'] = combinedROs['Amount'].astype(float)
mcadcsv['Amount'] = mcadcsv['Amount'].astype(float)
mcad_prep['Amount'] = mcad_prep['Amount'].astype(float)
agedROs['Amount'] = agedROs['Amount'].astype(float)
to_agedROs['Amount'] = to_agedROs['Amount'].astype(float)




#Post-time to hh:mm


combinedROs['Post-Time'] = pd.to_timedelta(combinedROs['Post-Time'],unit='S')
mcadcsv['Post-Time'] = pd.to_timedelta(mcadcsv['Post-Time'],unit='S')
mcad_prep['Post-Time'] = pd.to_timedelta(mcad_prep['Post-Time'],unit='S')
agedROs['Post-Time'] = pd.to_timedelta(agedROs['Post-Time'],unit='S')
to_agedROs['Post-Time'] = pd.to_timedelta(to_agedROs['Post-Time'],unit='S')



#all other columns
combinedROs[strcolumns] = combinedROs[strcolumns].astype(str)
mcadcsv[strcolumns] = mcadcsv[strcolumns].astype(str)
mcad_prep[strcolumns] = mcad_prep[strcolumns].astype(str)
agedROs[strcolumns] = agedROs[strcolumns].astype(str)
to_agedROs[strcolumns] = to_agedROs[strcolumns].astype(str)

In [41]:
#Today
practicedate = datetime.datetime.now() + datetime.timedelta(days=1)
date2y = practicedate - datetime.timedelta(days=730)
date10m1d = practicedate - datetime.timedelta(days=301)
date10m = practicedate - datetime.timedelta(days=300)
date9m = practicedate - datetime.timedelta(days=270)
date9m = practicedate - datetime.timedelta(days=270)
date7m = practicedate - datetime.timedelta(days=210)
date6m = practicedate - datetime.timedelta(days=180)
date120 = practicedate - datetime.timedelta(days=120)
date90 = practicedate - datetime.timedelta(days=90)
date60 = practicedate - datetime.timedelta(days=60)
date30 = practicedate - datetime.timedelta(days=30)
#print(date30,'\n',date60,'\n',date90,'\n',date6m,'\n',date7m,'\n',date9m,)
#practicedate>date7m
#min(date30, date60, date90, date6m, date9m)
print('21200', date120,'\n',date9m,)

21200 2020-02-20 19:20:36.005552 
 2019-09-23 19:20:36.005552


In [42]:
#mcadcsv[mcadcsv['Control1']=='109050']

In [43]:
#formated in relation to store and account
store_accounts = []
for a,b in schedules.items():
    for c in b:
        store_accounts.append((a,c))
#store_accounts

def dict_filter(Compss,Accountss):
    if (Compss,Accountss) in store_accounts:
        return True
        
mcadcsv['Dict'] = mcadcsv.apply(lambda x: dict_filter(x['Comp'],x['Account']),axis=1)
mcadcsv = mcadcsv[mcadcsv['Dict']==True].reset_index(drop=True)
mcadcsv = mcadcsv.drop(columns='Dict')


In [44]:
#mcadcsv[mcadcsv['Control1']=='109050']

In [45]:
#Summary Tab
mcad_summary = mcadcsv.groupby(Comp_Acct_Ctrl).agg({'Amount':['sum',],'Date':'min'}).round(2)
mcad_summary.columns = mcad_summary.columns.get_level_values(0)
mcad_summary['Days'] = (practicedate - mcad_summary['Date']).dt.days
mcad_summary = mcad_summary.reset_index(drop=False)#secures df by adding groupby columns back in df
mcad_summary = mcad_summary[(mcad_summary['Amount']!=0)&(mcad_summary['Days']<300)]
mcad_summary = mcad_summary.sort_values(by='Control1', ascending=True).reset_index(drop=True)

mcad_summary_id = mcad_summary[Comp_Acct_Ctrl].reset_index(drop=True)###comp,account,control1
mcad_summary_id = pd.merge(mcad_summary_id,archiveROs,on=Comp_Acct_Ctrl,how="outer",indicator=True)

In [46]:
mcad_summary[mcad_summary['Control1']=='109050']

,Comp,Account,Control1,Amount,Date,Days


In [47]:
#keep anything that is mcad_summary acct,account,control1

for i in mcad_summary_id.index:
    add = mcadcsv[(mcadcsv[Comp_Acct_Ctrl]==mcad_summary_id.loc[i,Comp_Acct_Ctrl]).all(axis=1)]
    mcad_prep = pd.concat([mcad_prep,add],sort=False,ignore_index=True).reset_index(drop=True)

In [48]:
mcad_prep[mcad_prep['Control1']=='109050']

,Comp,Account,Jrnl,Reference,Date,Amount,Control1,Post-Date,Post-Time,User


In [49]:
combinedROs[combinedROs['Control1']=='109050']

,Comp,Account,Jrnl,Reference,Date,Amount,Control1,Post-Date,Post-Time,User


In [50]:
newmcadcsv = mcad_prep.copy()
newmcadcsv1 = mcadcsv.copy()

intersectmcad = pd.merge(agedROs,newmcadcsv, on=thecolumns, how='inner', indicator=True).drop_duplicates().reset_index(drop=True)
#intersectmcad = pd.merge(agedROs,newmcadcsv, on=thecolumns, how='inner', indicator=True).reset_index(drop=True)

for j in intersectmcad.index:
    newmcadcsvdrop  = newmcadcsv[(newmcadcsv==intersectmcad.loc[j,thecolumns]).all(axis=1)]
    newmcadcsv = newmcadcsv.drop(newmcadcsvdrop.index)

combinedROs = pd.concat([agedROs,newmcadcsv],sort=False,ignore_index=True).reset_index(drop=True)

In [51]:
#newmcadcsv[newmcadcsv['Control1']=='109050']

In [52]:
#agedROs[(agedROs['Control1']=='109050')|(agedROs['Control1']=='31021')]

In [53]:
#combinedROs[(combinedROs['Control1']=='109050')|(combinedROs['Control1']=='31021')]

In [54]:
#Is this still required?
        
#combinedROs['Dict'] = combinedROs.apply(lambda x: dict_filter(x['Comp'],x['Account']),axis=1)
#combinedROs = combinedROs[combinedROs['Dict']==True].reset_index(drop=True)
#combinedROs = combinedROs.drop(columns='Dict')

In [55]:
#Summary Tab
summary = combinedROs.groupby(Comp_Acct_Ctrl).agg({'Amount':['sum',],'Date':'min'}).round(2)
summary.columns = summary.columns.get_level_values(0)
summary['Days'] = (practicedate - summary['Date']).dt.days
summary = summary.reset_index(drop=False)#secures df by adding groupby columns back in df

summary = summary.sort_values(by='Control1', ascending=True).reset_index(drop=True)

In [56]:
def age_col_91(dayss,sumss):
    if dayss > 90:
        return sumss
    else:
        return 0
def age_col_90(dayss,sumss):
    if 60 < dayss <= 90:
        return sumss
    else:
        return 0
def age_col_60(dayss,sumss):
    if 30 < dayss <= 60:
        return sumss
    else:
        return 0
def age_col_30(dayss,sumss):
    if dayss <= 30:
        return sumss
    else:
        return 0

summary['Current'] = summary.apply(lambda x: age_col_30(x['Days'],x['Amount']),axis=1)
summary['31-60 days'] = summary.apply(lambda x: age_col_60(x['Days'],x['Amount']),axis=1)
summary['61-90 days'] = summary.apply(lambda x: age_col_90(x['Days'],x['Amount']),axis=1)
summary['91+'] = summary.apply(lambda x: age_col_91(x['Days'],x['Amount']),axis=1)

In [57]:
#Select ROs (the oldest entry/Date:'min') days greater than aged 61 to add to agedROs
summary_aged = summary[summary['Days']>=61]
summary_aged = summary_aged[Comp_Acct_Ctrl].reset_index(drop=True)

In [58]:
#adding all comp, account, control where Days>91 as agedROs
#grab df from combinedROs that appears on summary_aged
for i in summary_aged.index:
    ageROs_add = combinedROs[(combinedROs[Comp_Acct_Ctrl]==summary_aged.loc[i,Comp_Acct_Ctrl]).all(axis=1)]
    to_agedROs = pd.concat([to_agedROs,ageROs_add],sort=False,ignore_index=True).reset_index(drop=True)

#find duplicated lines
intersectAgedROs = pd.merge(agedROs,to_agedROs,on=thecolumns,how="inner",indicator=True).drop_duplicates().reset_index(drop=True)
#intersectAgedROs = pd.merge(agedROs,to_agedROs,on=thecolumns,how="inner",indicator=True).reset_index(drop=True)


for j in intersectAgedROs.index:
    to_agedROsdrop  = to_agedROs[(to_agedROs == intersectAgedROs.loc[j,thecolumns]).all(axis=1)]
    to_agedROs = to_agedROs.drop(to_agedROsdrop.index)
    
agedROs = pd.concat([agedROs,to_agedROs],sort=False,ignore_index=True).reset_index(drop=True)

In [59]:
#find intercepts between agedROs and newmcadcsv1, remove intercepts from newmcadcsv1
#intersectAgedROs_mcad = pd.merge(agedROs,newmcadcsv1,on=thecolumns,how="inner",indicator=True).drop_duplicates().reset_index(drop=True)
intersectAgedROs_mcad = pd.merge(agedROs,newmcadcsv1,on=thecolumns,how="inner",indicator=True).reset_index(drop=True)

for q in intersectAgedROs_mcad.index:
    drop1 = newmcadcsv1[(newmcadcsv1==intersectAgedROs_mcad.loc[q,thecolumns]).all(axis=1)]
    newmcadcsv1 = newmcadcsv1.drop(drop1.index)
newmcadcsv1 = newmcadcsv1.reset_index(drop=True)


In [60]:
newmcadcsv1[newmcadcsv1['Control1']=='109050']

,Comp,Account,Jrnl,Reference,Date,Amount,Control1,Post-Date,Post-Time,User


In [61]:

#add comp,account,control1 to agedROs from newmcadcsv1 
intersectAgedROs_mcad_CompAcctCtrl = agedROs.groupby(Comp_Acct_Ctrl).agg({'Amount':['sum',],}).round(2)
intersectAgedROs_mcad_CompAcctCtrl.columns = intersectAgedROs_mcad_CompAcctCtrl.columns.get_level_values(0)
intersectAgedROs_mcad_CompAcctCtrl.reset_index(level=['Comp','Account','Control1'],inplace=True)
for k in intersectAgedROs_mcad_CompAcctCtrl.index:
    add1 = newmcadcsv1[(newmcadcsv1[Comp_Acct_Ctrl]==intersectAgedROs_mcad_CompAcctCtrl.loc[k,Comp_Acct_Ctrl]).all(axis=1)]
    agedROs = pd.concat([agedROs,add1],sort=False,ignore_index=True).reset_index(drop=True)


In [62]:
#from ageROs, selected to drop where sum totals to zero and RO Date (youngest entry/Date:'max') older than 2 years
agedROs1 = agedROs.groupby(Comp_Acct_Ctrl).agg({'Amount':['sum',],'Date':'max'}).round(2)
agedROs1.columns = agedROs1.columns.get_level_values(0)
agedROs1 = agedROs1.reset_index(drop=False)
agedROs1_drop = agedROs1[(agedROs1['Amount']==0)&(agedROs1['Date']<date10m1d)].reset_index(drop=True)

for r in agedROs1_drop.index:
    drop = agedROs[(agedROs[Comp_Acct_Ctrl]==agedROs1_drop.loc[r,Comp_Acct_Ctrl]).all(axis=1)]
    agedROs = agedROs.drop(drop.index)      

agedROs = agedROs.reset_index(drop=True)

In [63]:
agedROs[(agedROs['Control1']=='109050')|(agedROs['Control1']=='31021')]

,Comp,Account,Jrnl,Reference,Date,Amount,Control1,Post-Date,Post-Time,User
1562,41,26300,35,31021,2020-01-13,1356.69,31021,2020-01-13,20:12:07,"LEVESQUE,KATIE"
1586,33,26300,35,109050,2020-01-30,1585.17,109050,2020-01-30,20:45:09,"MARCIELLO,ELIJAH"
2159,33,26300,90,227-041720,2020-04-17,43.92,109050,2020-04-23,15:15:42,LAWRENCEXIE
2426,41,26300,90,227-042720,2020-04-27,-59.37,31021,2020-04-27,16:05:59,LAWRENCEXIE
4985,41,26300,58,00118,2020-01-17,-283.02,31021,2020-01-23,13:33:51,LAWRENCEXIE
4986,41,26300,58,00125,2020-01-24,-865.88,31021,2020-01-29,12:47:00,LAWRENCEXIE
4987,41,26300,58,00321,2020-03-20,-148.42,31021,2020-03-26,15:56:03,LAWRENCEXIE
5270,33,26300,58,2020-01-31,2020-01-31,-37.20,109050,2020-02-03,11:13:19,LAWRENCEXIE
5271,33,26300,58,2020-01-31,2020-01-31,-37.20,109050,2020-02-03,11:13:19,LAWRENCEXIE
5272,33,26300,58,2020-01-31,2020-01-31,-148.88,109050,2020-02-03,11:13:19,LAWRENCEXIE


In [64]:
combinedROs[(combinedROs['Control1']=='109050')|(combinedROs['Control1']=='31021')]

,Comp,Account,Jrnl,Reference,Date,Amount,Control1,Post-Date,Post-Time,User
1562,41,26300,35,31021,2020-01-13,1356.69,31021,2020-01-13,20:12:07,"LEVESQUE,KATIE"
1586,33,26300,35,109050,2020-01-30,1585.17,109050,2020-01-30,20:45:09,"MARCIELLO,ELIJAH"
2159,33,26300,90,227-041720,2020-04-17,43.92,109050,2020-04-23,15:15:42,LAWRENCEXIE
2426,41,26300,90,227-042720,2020-04-27,-59.37,31021,2020-04-27,16:05:59,LAWRENCEXIE
4985,41,26300,58,00118,2020-01-17,-283.02,31021,2020-01-23,13:33:51,LAWRENCEXIE
4986,41,26300,58,00125,2020-01-24,-865.88,31021,2020-01-29,12:47:00,LAWRENCEXIE
4987,41,26300,58,00321,2020-03-20,-148.42,31021,2020-03-26,15:56:03,LAWRENCEXIE
5270,33,26300,58,2020-01-31,2020-01-31,-37.20,109050,2020-02-03,11:13:19,LAWRENCEXIE
5271,33,26300,58,2020-01-31,2020-01-31,-37.20,109050,2020-02-03,11:13:19,LAWRENCEXIE
5272,33,26300,58,2020-01-31,2020-01-31,-148.88,109050,2020-02-03,11:13:19,LAWRENCEXIE


In [65]:
###save

agedROs = agedROs.dropna().reset_index(drop=True)
agedROs_str = agedROs.astype('str')
#agedROs.to_csv(r'C:\Users\L\Desktop\RO Schedules\agedROs\agedROs.csv',index=False)###home
agedROs_str.to_csv(r'C:\Users\lxie\Desktop\Python Warranty\agedROs.csv',index=False)###work

In [66]:
#file prep
sch_details = pd.DataFrame(columns=thecolumns)
sch_summary = summary[summary_columns]
sch_summary = sch_summary[sch_summary['Amount']!=0]
sch_summary = sch_summary.sort_values(by='Control1',ascending=True).reset_index(drop=True)

In [67]:
sch_summary1 = sch_summary.copy() #sch_summary1 with combinedROs3 to help prepare sch_details
sch_summary1 = sch_summary1[Comp_Acct_Ctrl]
combinedROs3 = combinedROs.copy()

In [68]:
#select only comp,account,Controls that are relevant
for m in sch_summary1.index:
    add2 = combinedROs3[(combinedROs3[Comp_Acct_Ctrl]==sch_summary1.loc[m,Comp_Acct_Ctrl]).all(axis=1)]
    sch_details = pd.concat([sch_details,add2],sort=False,ignore_index=True).reset_index(drop=True)

In [69]:
#sch_details['Date'] = pd.to_datetime(sch_details['Date'],format='%Y%m%d')

In [70]:
#sch_details

In [71]:
sch_details1 = pd.concat([sch_details,sch_summary],sort=False,ignore_index=True)
sch_details1 = sch_details1[detail_columns]
sch_details1 = sch_details1.sort_values(by=['Control1','Date'], ascending=True).reset_index(drop=True)

In [72]:
#sch_details1['Date'] = pd.to_datetime(sch_details1['Date'],format='%Y%m%d',errors='ignore')

In [73]:
#sch_details1

In [74]:
sch_details1 = sch_details1.fillna('')

In [75]:
#os.chdir(r'C:\Users\L\Desktop\RO Schedules\Email Schedules')###home

os.chdir(r'C:\Users\lxie\Desktop\Python Warranty\email schedules')###work

for i in store_accounts:
    store = i[0]
    account = i[1]
    
    sch_summary2 = sch_summary[(sch_summary['Comp']==store)&(sch_summary['Account']==account)]
    sch_summary2.sort_values(by='Control1', ascending=True).reset_index(drop=True)
    sch_details2 = sch_details1[(sch_details1['Comp']==store)&(sch_details1['Account']==account)]
    sch_details2.sort_values(by='Control1', ascending=True).reset_index(drop=True)
    
    summmary_days = sch_summary2['Days'].max()#####add Days######
    current_total = sch_summary2['Current'].sum().round(2)
    sixty_total = sch_summary2['31-60 days'].sum().round(2)
    ninety_total = sch_summary2['61-90 days'].sum().round(2)
    ninetyone_total = sch_summary2['91+'].sum().round(2)
    grand_total = sch_summary2['Amount'].sum().round(2)
    
    
    total_list  = ['Grand Total',grand_total,current_total,sixty_total,ninety_total,ninetyone_total]
    total_df = pd.DataFrame([total_list], columns=total_columns)
    
    sch_summary2 = sch_summary2.append(total_df,ignore_index=True,sort=False)
    sch_details2 = sch_details2.append(total_df,ignore_index=True,sort=False)
    sch_summary2.fillna('')
    sch_summary2 = sch_summary2.replace(0,'')
    sch_details2.fillna('')
    sch_details2 = sch_details2.replace(0,'')
    
    
    
    with pd.ExcelWriter("{}-{}.xlsx".format(store,account)) as writer:
        sch_details2.to_excel(writer, sheet_name='Details',index=False)
        sch_summary2.to_excel(writer, sheet_name='Summary',index=False)
    
    print(store,account,summmary_days,total_list)
    


16 26300 119 ['Grand Total', 14919.15, 0.0, 0.0, 0.0, 14919.15]
16 26410 92 ['Grand Total', 1180.17, 0.0, 0.0, 845.23, 334.94]
16 26310 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
16 26330 197 ['Grand Total', -486.86, 0.0, 0.0, -232.64, -254.22]
16 26320 730 ['Grand Total', -2302.31, 0.0, 0.0, -378.96, -1923.35]
17 26300 108 ['Grand Total', 563.95, 0.0, 0.0, 0.0, 563.95]
17 26410 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
17 26320 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
18 26300 104 ['Grand Total', 2433.2, 0.0, 0.0, 0.0, 2433.2]
18 26410 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
18 26420 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
18 26320 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
19 26410 67 ['Grand Total', -60.0, 0.0, 0.0, -60.0, 0.0]
19 26420 219 ['Grand Total', -500.0, 0.0, 0.0, 0.0, -500.0]
20 26300 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
20 26410 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
20 26320 nan ['Grand Total', 0.0, 0.0, 0.0, 0.0, 0.0]
23 26300 107 ['G